## Preparing to Train a Model

We can use the unlabelled data this time as this gives us an extra 50000 additional reviews with no labels.

In [1]:
import pandas as pd

# Read data from files
path = "/home/sophie/projects/kaggleBOW/"
train = pd.read_csv("%slabeledTrainData.tsv"%path, header=0, delimiter="\t", quoting=3)
test = pd.read_csv("%stestData.tsv"%path, header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv("%sunlabeledTrainData.tsv"%path, header=0, delimiter="\t", quoting=3)

In [2]:
print(list(test))

['id', 'review']


In [3]:
# verify the number of reviews that were read(100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews \
      and %d unlabeled reviews\n," %(train["review"].size,test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews       and 50000 unlabeled reviews
,


To train Word2Vec it might not be better to remove stop words, as these give us information about the broader context of the sentence. It is also possible to leave numbers in too. 

In [4]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review, remove_stopwords=False):
    """Convert a document to a sequence of words, optionally removing stop words
    Returns a list of words"""
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    # 5. Return a list of words
    return(words)

Word2Vec expects single sentences, each as a list of words. i.e. a list of lists.
We will use NLTKs punkt tokenizer for sentence splitting. 

In [5]:
# Download the punkt tokenizer for sentence splitting
import nltk.data

#Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences(review, tokenizer, remove_stopwords=False):
    """Split a review into parsed sentences. Returns a list of sentences,
    where each sentence is a list of words"""
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())

    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
            
    # Return the list of sentences (each sentence is a list of words, so this returns a list of lists)
    return sentences


Now we can apply this function to prepare our data for input to Word2Vec (takes a couple of mins)

In [6]:
sentences = [] # initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)
    
print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/home/sophie/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/home/sophie/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:198: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/home/sophie/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:207: UserWarning: "b'http://www.happierabroad.com"'" looks like a URL. Beautiful So

Parsing sentences from unlabeled set


/home/sophie/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:207: UserWarning: "b'http://www.archive.org/details/LovefromaStranger"'" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/home/sophie/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:207: UserWarning: "b'http://www.loosechangeguide.com/LooseChangeGuide.html"'" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful So

In [14]:
# Check how many sentences we have in total - should be around 850 000
print(len(sentences))

print(sentences[2])

795538
['moonwalker', 'is', 'part', 'biography', 'part', 'feature', 'film', 'which', 'i', 'remember', 'going', 'to', 'see', 'at', 'the', 'cinema', 'when', 'it', 'was', 'originally', 'released']


### Training and Saving Your Model

Now we have a list of nicely parse sentences, we're ready to train the model. There are a number of parameter choices that affect the run time and quality fo the final model that is produced.

- Architecture
- Training algorithm
- Downsampling of frequent words
- Word vector dimensionality
- Context / window size
- Worker threads
- Minimum word count

Choosing parameters is not easy, but once we have chosen our parameters, creating Word2Vec model is straightforward.

In [16]:
# Import the built-in logging module and configure it so that Word2Vec creates nice output messages
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s, level = logging.INFO ')

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                 
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model....")
model = word2vec.Word2Vec(sentences, workers = num_workers, size = num_features, min_count = min_word_count
                          , window = context, sample = downsampling)

# If you don't plan to train the model any further, calling init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and save the model for later use. You can load it later using
# Word2Vec.load()
model_name = "/home/sophie/projects/kaggleBOW/models/300features_40minwords_10context"
model.save(model_name)


Training model....


### Exploring the Model Results

The "doesnt_match" function will try to deduce which word in a set is most dissimilar from the others

In [17]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

Our model is capable of distinguishing differences in meaning! 

In [18]:
model.doesnt_match("france england germany berlin".split())

'berlin'

In [19]:
model.doesnt_match("paris berlin london austria".split())

'paris'

Doesn't quite always get it.

We can also use the `most_similar` function to get insight into the model's word clusters

In [20]:
model.most_similar("man")

[('woman', 0.6190699338912964),
 ('lady', 0.5802098512649536),
 ('lad', 0.5688491463661194),
 ('monk', 0.5366647839546204),
 ('chap', 0.5279662609100342),
 ('guy', 0.5215261578559875),
 ('men', 0.5197211503982544),
 ('sailor', 0.512083888053894),
 ('soldier', 0.5092530846595764),
 ('person', 0.5068249106407166)]

In [21]:
model.most_similar("queen")

[('princess', 0.663836658000946),
 ('bride', 0.6022839546203613),
 ('maid', 0.5899404883384705),
 ('goddess', 0.5886632800102234),
 ('mistress', 0.5708042979240417),
 ('prince', 0.5670790672302246),
 ('eva', 0.5657265186309814),
 ('victoria', 0.5646870732307434),
 ('kristel', 0.5638813376426697),
 ('stepmother', 0.5635300278663635)]

Some more relevent sentiment analysis

In [22]:
model.most_similar("awful")

[('terrible', 0.76213538646698),
 ('horrible', 0.7376803159713745),
 ('dreadful', 0.7240302562713623),
 ('abysmal', 0.7220585942268372),
 ('atrocious', 0.7205259203910828),
 ('horrendous', 0.6998444199562073),
 ('appalling', 0.6866902112960815),
 ('horrid', 0.6556534171104431),
 ('amateurish', 0.6206444501876831),
 ('laughable', 0.6163826584815979)]

So, now we have a reasonably good model for semantic meaning - at least as good as Bag of Words. How can we use these fancy distributed word vectors for supervised learning. See next section..